In [34]:
import pandas as pd
import numpy as np
import requests
import time
import datetime as dt
import pymongo
import ast
import json
from bson.json_util import dumps
from pprint import pprint

In [39]:
add = "bjs/ncvs/v2/household/2010"
add = "bjs/ncvs/v2/household/fields/"
query_url = f'https://api.bjs.ojp.gov/{add}?format=json'

response = requests.get(query_url)

In [40]:
pprint(response.text)


('<?xml version="1.0" encoding="UTF-8" '
 'standalone="yes"?><bjsDataSetFields><fields><description>Household '
 'victimization includes all property victimization (household burglary, motor '
 'vehicle theft, and theft). This category includes both attempted and '
 'completed crimes.</description><id>newcrime</id><name>Aggregate type of '
 'crime</name><values><description>Property '
 'victimization</description><name>3</name></values></fields><fields><description>The '
 "respondent's age on the last day of the month before the interview.  The "
 'NCVS collects information on household members age 12 or older.  The age of '
 'the head of household is used for computing household crime '
 'demographics.</description><id>hhage</id><name>Head of household '
 'age</name><values><description>19 or '
 'younger</description><name>1</name></values><values><description>20 to '
 '34</description><name>2</name></values><values><description>35 to '
 '49</description><name>3</name></values><values

In [38]:
import xmltodict, json

o = xmltodict.parse('<e> <a>text</a> <a>text</a> </e>')
json.dumps(o)

ModuleNotFoundError: No module named 'xmltodict'